# Load Data

In [1]:
# Imports
# from settings import *
# import analyze_cascade
import datetime
import pandas as pd
import numpy as np
from itertools import groupby
from collections import Counter
from random import shuffle
from random import choices
import math
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

/Users/nic/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
metadata_file = 'metadata_anon.txt'

In [3]:
# Read meta data 
fin = open(metadata_file,'r')
lines = fin.readlines()
fin.close()
cascade_id2metadata={}
for line in lines:
    line = line.replace('\n','')
    item = eval(line)
    cascade_id2metadata[item[0]] = item[1]

## Descriptives of dynamic measures

### Static measures

In [33]:
# Get static measures
veracity = []
virality = []
depth = []
breadth = []
size = []
verified = []
nfollowers = []
nfollowees = []
engagement = []
category = []
for cascade,metadata in cascade_id2metadata.items():
    if metadata['virality'] is not None: 
        veracity.append(metadata['veracity'])
        virality.append(metadata['virality'])
        depth.append(metadata['depth'])
        breadth.append(metadata['max_breadth'])
        size.append(metadata['size'])
        verified.append(metadata['verified_list'][0])
        nfollowers.append(metadata['num_followers_list'][0])
        nfollowees.append(metadata['num_followees_list'][0])
        engagement.append(metadata['engagement_list'][0])
        category.append(metadata['rumor_category'])

# Convert to data frame
df = pd.DataFrame({'veracity': veracity, 'virality': virality, 'depth': depth, 'breadth': breadth, 'size': size, 'verified': verified, 'nfollowers': nfollowers, 
                   'nfollowees': nfollowees, 'engangement': engagement, 'category': category})

# Inspect
df.head(5)

,breadth,category,depth,engangement,nfollowees,nfollowers,size,veracity,verified,virality
0,10703,Viral Photos/Stories/Urban Legends,11,25.799399,186.0,672.0,23228,MIXED,False,4.003857
1,11783,Science/Nature/Tech/Food/Health,9,10.811974,313.0,380.0,14827,MIXED,False,2.535338
2,6504,Viral Photos/Stories/Urban Legends,13,15.395237,518.0,504.0,14129,MIXED,False,4.019705
3,5772,Viral Photos/Stories/Urban Legends,8,3.140842,189.0,228.0,9972,MIXED,False,3.271008
4,6041,Viral Photos/Stories/Urban Legends,8,5.160261,174.0,110.0,9526,MIXED,False,3.115942


### Dynamic measures

In [34]:
len_depth2time = []
len_num_followees_list = []
len_depth2uu = []
len_uu2time = []
len_depth2breadth = []
for cascade,metadata in cascade_id2metadata.items():
    if metadata['virality'] is not None: 
        len_depth2time.append(len(metadata['depth2time'].keys()))
        len_num_followees_list.append(len(metadata['num_followees_list']))
        len_depth2uu.append(len(metadata['depth2uu'].keys()))
        len_uu2time.append(len(metadata['uu2time'].keys()))
        len_depth2breadth.append(len(metadata['depth2breadth'].keys()))
    
# Convert to data frame
df_len = pd.DataFrame({'depth2time ': len_depth2time, 
                       'num_followees_list': len_num_followees_list, 
                       'depth2uu': len_depth2uu, 
                       'uu2time': len_uu2time, 
                       'depth2breadth': len_depth2breadth})

# # Get summary
df_len.describe(percentiles = [0.25, 0.5, 0.75, 1])
    

,depth2breadth,depth2time,depth2uu,num_followees_list,uu2time
count,42081.000000,42081.000000,42081.000000,42081.000000,42081.000000
mean,1.707707,1.707707,1.707707,93.878829,93.878829
std,1.319555,1.319555,1.319555,950.694376,950.694376
min,1.000000,1.000000,1.000000,2.000000,2.000000
25%,1.000000,1.000000,1.000000,2.000000,2.000000
50%,1.000000,1.000000,1.000000,4.000000,4.000000
75%,2.000000,2.000000,2.000000,9.000000,9.000000
100%,24.000000,24.000000,24.000000,46895.000000,46895.000000
max,24.000000,24.000000,24.000000,46895.000000,46895.000000


## Create LSTM data

### Get dynamic data

In [8]:
# Function to get expression of each item in a dictionary entry
def get_expression_list(entry):
    expression = []
    for i in entry.keys():
        expression.append(float(entry[i]))
    return expression

# Convert y to classification
def veracity_to_categorical(v):
    if v == 'FALSE':
        vbin = [1,0,0]
    elif v == 'MIXED':
        vbin = [0,1,0]
    elif v == 'TRUE':
        vbin = [0,0,1]
    return vbin

# Get data in list format
data = []
for cascade,metadata in cascade_id2metadata.items():
    if metadata['virality'] is not None:       
        # Get depth
        depth2time = get_expression_list(metadata['depth2time'])
        depth2uu = get_expression_list(metadata['depth2uu'])
        depth2breadth = get_expression_list(metadata['depth2breadth']) 
        veracity = veracity_to_categorical(metadata['veracity'])
        data_id = []
        for time, uu, breadth in zip(depth2time, depth2uu, depth2breadth):
            data_t = [cascade, 
                      veracity,
                      time, uu, breadth]
            data_id.append(data_t)
        data.extend([data_id])

### Preprocessing

In [ ]:
# Function: Create training and test set
def split_list(lst, train_size): # train_size is a proportion
    split = len(lst) * train_size
    if split.is_integer():
        split = int(split)
        return lst[:split], lst[split:]
    else:
        split = math.floor(split) + 1
        return lst[:split], lst[split:]
    
# Function: Padding for groups of equal batches
def padding(lst, bsize):
    if len(lst) % bsize != 0:
        psize = bsize - (len(lst) % 5)
        samples = choices(lst, k=psize)
        lst.extend(samples)
    return lst

# Get sublist
def get_sublist(list_in_list, start, stop):
    x = []
    for lst in list_in_list:
        x_id = []
        for sublist in lst:
            if stop is None:
                x_id.append(sublist[start:])
            elif start is None:
                x_id.append(sublist[:stop])
            else:
                x_id.append(sublist[start:stop])
        x.extend([x_id])
    return x

# Separate id, x and y
def separate(list_in_list):
    cid = []
    y = []
    for lst in list_in_list:
        cid.append(lst[0][0]) # only one id is needed
#         veracity_id = []
#         for sublist in lst:
#             veracity_id.extend([sublist[1]])
#         veracity.append(veracity_id)
        y.append(lst[0][1])
    x = get_sublist(list_in_list,2,None)
    return cid, y, x

# # Group by sequence length and append to have batches of 5 for both training and test
data.sort(key=len)   # Randomly reshuffle before? random.shuffle(...)
x_train = []
x_test = []
y_train = []
y_test = []
cid_train = []
cid_test = []
for k, g in groupby(data, len):
    group = list(g)
    if len(group) > 2: # This omits too small groups
        shuffle(group)
        # Create train and test bucket
        train_group, test_group = split_list(group, 0.5)
        # Padd for equal batch size
        train_group_padded = padding(train_group, 5)
        test_group_padded = padding(test_group, 5)
        # Separate list
        cid_train_group, y_train_group, x_train_group = separate(train_group)
        cid_test_group, y_test_group, x_test_group = separate(test_group)
        # Append:  convert y and x into numpy array
        x_train.append(np.array(x_train_group))
        x_test.append(np.array(x_test_group))
        y_train.append(np.array(y_train_group))
        y_test.append(np.array(y_test_group))
        cid_train.append(cid_train_group)
        cid_test.append(cid_test_group)

### Data standardization

In [ ]:
# Function to standardize the list
def standardization(lst, index, mean, std):
    for array3d in lst:
        for array2d in array3d:
            for vector in array2d:
                vector[index] = (vector[index] - mean) / std
    return lst

# Function to compute mean and std of variable and then standardizes this variable in list
def standardize_data(a_list, b_list, index):
    var = []
    # Compute mean and std from train data variable
    for array3d in a_list:
        for array2d in array3d:
            for vector in array2d:
                var.append(vector[index])
    var = np.array(var)
    var_mean = var.mean()
    var_std = var.std()
    # Standardize a
    a_list_std = standardization(a_list, index, var_mean, var_std)
    b_list_std = standardization(b_list, index, var_mean, var_std)
    return a_list_std, b_list_std

# Standardize all variables
def standardize_all(a_list, b_list):
    length = len(a_list[0][0][0])
    indices = list(range(length))
    for i in indices:
        std_a, std_b = standardize_data(a_list, b_list, i)
    return std_a, std_b

x_train, x_test = standardize_all(x_train, x_test)

## LSTM train data descriptives

In [20]:
# Group size and sequence length
i = 1
for g in x_train:
    print('Group: ', i, ' ', 'Observations: ', len(g), ' ' 'Sequence length', len(g[0]))
    i += 1

Group:  1   Observations:  12960  Sequence length 1
Group:  2   Observations:  4790  Sequence length 2
Group:  3   Observations:  1790  Sequence length 3
Group:  4   Observations:  760  Sequence length 4
Group:  5   Observations:  320  Sequence length 5
Group:  6   Observations:  170  Sequence length 6
Group:  7   Observations:  95  Sequence length 7
Group:  8   Observations:  65  Sequence length 8
Group:  9   Observations:  35  Sequence length 9
Group:  10   Observations:  25  Sequence length 10
Group:  11   Observations:  20  Sequence length 11
Group:  12   Observations:  15  Sequence length 12
Group:  13   Observations:  10  Sequence length 13
Group:  14   Observations:  10  Sequence length 14
Group:  15   Observations:  5  Sequence length 15
Group:  16   Observations:  5  Sequence length 16
Group:  17   Observations:  5  Sequence length 17
Group:  18   Observations:  5  Sequence length 19


In [32]:
# Convert y to classification
def reverse_veracity_to_categorical(vbin):
    if vbin[0] == 1:
        v = 'FALSE'
    elif vbin[1] == 1:
        v = 'MIXED'
    elif vbin[2] == 1:
        v = 'TRUE'
    return v

# Outcome distribution
i = 1
for g in y_train:
    ver = []
    for y in g:
        ver.append(reverse_veracity_to_categorical(y))
    print('Group: ', i, Counter(ver))
    i += 1

Group:  1 Counter({'FALSE': 9665, 'TRUE': 2010, 'MIXED': 1285})
Group:  2 Counter({'FALSE': 3669, 'TRUE': 675, 'MIXED': 446})
Group:  3 Counter({'FALSE': 1375, 'TRUE': 248, 'MIXED': 167})
Group:  4 Counter({'FALSE': 608, 'TRUE': 90, 'MIXED': 62})
Group:  5 Counter({'FALSE': 245, 'TRUE': 50, 'MIXED': 25})
Group:  6 Counter({'FALSE': 127, 'TRUE': 24, 'MIXED': 19})
Group:  7 Counter({'FALSE': 81, 'MIXED': 7, 'TRUE': 7})
Group:  8 Counter({'FALSE': 52, 'MIXED': 8, 'TRUE': 5})
Group:  9 Counter({'FALSE': 30, 'TRUE': 4, 'MIXED': 1})
Group:  10 Counter({'FALSE': 21, 'TRUE': 2, 'MIXED': 2})
Group:  11 Counter({'FALSE': 20})
Group:  12 Counter({'FALSE': 14, 'TRUE': 1})
Group:  13 Counter({'FALSE': 7, 'MIXED': 3})
Group:  14 Counter({'FALSE': 9, 'MIXED': 1})
Group:  15 Counter({'FALSE': 5})
Group:  16 Counter({'FALSE': 5})
Group:  17 Counter({'FALSE': 5})
Group:  18 Counter({'FALSE': 3, 'MIXED': 2})


## LSTM on dynamic measures

In [12]:
# Create LSTM model
model = Sequential()
model.add(LSTM(5, input_shape = (None, 3),  return_sequences = False))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Epoch 1/2
12960/12960 [==============================] - 14s 1ms/step - loss: 0.7598 - acc: 0.7485
Epoch 2/2
12960/12960 [==============================] - 12s 941us/step - loss: 0.7329 - acc: 0.7485
Epoch 1/2
4790/4790 [==============================] - 5s 1ms/step - loss: 0.6999 - acc: 0.7693
Epoch 2/2
4790/4790 [==============================] - 6s 1ms/step - loss: 0.6975 - acc: 0.7693
Epoch 1/2
1790/1790 [==============================] - 3s 2ms/step - loss: 0.7204 - acc: 0.7581
Epoch 2/2
1790/1790 [==============================] - 3s 1ms/step - loss: 0.7197 - acc: 0.7587
Epoch 1/2
760/760 [==============================] - 2s 2ms/step - loss: 0.6841 - acc: 0.7776
Epoch 2/2
760/760 [==============================] - 1s 2ms/step - loss: 0.6777 - acc: 0.7776
Epoch 1/2
320/320 [==============================] - 1s 2ms/step - loss: 0.6809 - acc: 0.7844
Epoch 2/2
320/320 [==============================] - 1s 2ms/step - loss: 0.6759 - acc: 0.7844
Epoch 1/2
170/170 [=====================

In [ ]:
# Fit model and get train predictions
train_pred = []
for X,Y in zip(x_train, y_train):
    hist = model.fit(X, Y, epochs=2, batch_size=5)
    pred = model.predict(X, batch_size=5)
    train_pred.append(pred)

In [32]:
# Convert predictions to data frame with ID
cid = []
y1 = []
y2 = []
y3 = []
for pred_group in train_pred:
    for pred in pred_group:
        y1.append(pred[0])
        y2.append(pred[1])
        y3.append(pred[2])
        
for cid_group in cid_train:
    for i in cid_group:
        cid.append(i)

ytrain_pred = pd.DataFrame({'cid': cid, 'y1': y1, 'y2': y2, 'y3': y3})
ytrain_pred = ytrain_pred.drop_duplicates('cid')

          cid        y1        y2        y3
0       67863  0.742731  0.099418  0.157852
1       55164  0.744828  0.098442  0.156730
2       57868  0.744828  0.098442  0.156730
3       79351  0.707033  0.116352  0.176615
4      110652  0.744828  0.098442  0.156730
5      108784  0.743257  0.099173  0.157571
6       62670  0.744305  0.098685  0.157010
7       58840  0.744828  0.098442  0.156730
8       88434  0.744828  0.098442  0.156730
9       50899  0.744828  0.098442  0.156730
10     108705  0.742731  0.099418  0.157852
11      62551  0.744305  0.098685  0.157010
12      68657  0.742204  0.099663  0.158133
13      85498  0.743781  0.098929  0.157290
14      86713  0.744828  0.098442  0.156730
15      68138  0.742731  0.099418  0.157852
16      51048  0.744828  0.098442  0.156730
17      54770  0.744828  0.098442  0.156730
18      65326  0.743781  0.098929  0.157290
19      64362  0.743781  0.098929  0.157290
20      63413  0.744305  0.098685  0.157010
21      59624  0.744828  0.09844